# Test getting club specific urls

In [8]:
import requests
from bs4 import BeautifulSoup
from web_scraping import web_table_to_dataframe, get_soccer_leagues_ids

In [3]:
response = requests.get("https://fbref.com/en/comps/20/Bundesliga-Stats")
soup = BeautifulSoup(response.content, "html.parser")

In [4]:
len(soup("table"))

24

In [6]:
web_table_to_dataframe(soup("table")[1])

,Rk,Squad,MP,W,D,L,GF,GA,GD,Pts,...,L,GF,GA,GD,Pts,Pts/MP,xG,xGA,xGD,xGD/90
0,1,Bayern Munich,7,5,2,0,26,6,+20,17,...,1,23,7,+16,17,2.13,16.9,7.9,+9.0,+1.12
1,2,Freiburg,7,5,1,1,12,5,+7,16,...,2,13,12,+1,14,1.75,8.5,11.2,-2.7,-0.34
2,3,RB Leipzig,7,6,1,0,19,5,+14,19,...,3,11,16,-5,9,1.13,10.3,11.2,-0.9,-0.11
3,4,Eint Frankfurt,8,4,1,3,18,13,+5,13,...,1,14,11,+3,14,2.00,10.0,8.8,+1.3,+0.18
4,5,Union Berlin,7,5,2,0,14,6,+8,17,...,4,10,14,-4,10,1.25,7.7,10.2,-2.6,-0.32
5,6,Dortmund,7,5,1,1,15,5,+10,16,...,5,10,16,-6,9,1.13,12.9,13.7,-0.8,-0.10
6,7,Wolfsburg,7,3,3,1,15,10,+5,12,...,3,9,10,-1,11,1.38,5.9,13.5,-7.6,-0.95
7,8,M'Gladbach,8,6,0,2,20,10,+10,18,...,3,8,14,-6,4,0.57,7.8,11.9,-4.0,-0.57
8,9,Werder Bremen,8,3,1,4,14,13,+1,10,...,2,11,14,-3,11,1.57,9.3,13.1,-3.8,-0.54
9,10,Mainz 05,7,1,4,2,8,9,-1,7,...,4,11,15,-4,12,1.50,10.5,14.7,-4.1,-0.52


In [ ]:
### extract all soccer data

from dotenv import load_dotenv
load_dotenv(override=True)
import os
from os.path import join
import sys
import time

from web_scraping import get_soccer_leagues_ids, get_soccer_league_season_standings, get_soccer_teams_url, get_player_season_stats

current_season = 2022
soccer_league_ids = get_soccer_leagues_ids()

for year in range(2000,2022):
    print(year)
    for soccer_league_id in soccer_league_ids:
        if soccer_league_id["league_name"] == "Fußball-Bundesliga" or soccer_league_id["league_name"] == "Big 5 European Leagues Combined":
            continue
        # get season standings
        df = get_soccer_league_season_standings(year, current_season, soccer_league_id["league_id"], soccer_league_id["league_stats_name"])
        df.to_csv(join(os.environ["SportsDB"], "soccer", f"{soccer_league_id['league_name'].replace(' ','-')}_season_standings_{year}.csv"), index=False)
        
        
        # get player stats
        df = get_player_season_stats(year, current_season, soccer_league_id["league_id"], soccer_league_id["league_stats_name"])
        df.to_csv(join(os.environ["SportsDB"], "soccer", f"{soccer_league_id['league_name'].replace(' ','-')}_player_stats_{year}.csv"), index=False)
    
        time.sleep(3)
    

# Extract column headers from different tables

In [16]:
import json
from glob import glob
from dotenv import load_dotenv
load_dotenv(override=True)

import os
from os.path import join
import sys
import pandas as pd

metadata = {}

tables = ["player_stats", "season_standings"]

for table in tables:
    column_names = []
    for found_table in glob(join(os.environ["SportsDB"], "soccer", f"*{table}_*.csv")):
        column_names.extend(list(pd.read_csv(found_table).columns))
    
    column_names = list(set(column_names))
    metadata[table] = {}
    for column_name in column_names:
        metadata[table][column_name] = None
    
if os.path.isfile("metadata.json"):
    print("metadata.json exists already!!! ")
else:
    with open("metadata.json", "w") as f:
        json.dump(metadata, f)

metadata.json exists already


# Extract valid header file

In [ ]:
import json
from glob import glob
import os 
os.environ["SPORTS_DB"] = "/ext/daten-wi/slangenecker/SportsDB"
from os.path import join
import pandas as pd

def get_sportsDB_soccer_type_mappings():
    with open(join(os.environ["SPORTS_DB"], "Soccer", "Soccer-Relations.json"), "r") as f:
        type_mappings = json.load(f)["soccer-big5Leagues-Players-2021-2022.csv"]["columns"][0]
        return type_mappings


def get_all_sportsDB_soccer_tables(only_file_names:bool=False):
    result = glob(join(os.environ["SPORTS_DB"], "Soccer", "soccerPlayerScraping", "*.csv"))
    if only_file_names:
        return [table.split("/")[-1] for table in result]
    else:
        return result

def load_sportsDB_soccer_table(table_name:str, only_headers:bool=False):
    df = pd.read_csv(join(os.environ["SPORTS_DB"], "Soccer", "soccerPlayerScraping", table_name))
    if only_headers:
        return df.columns
    else:
        return df

In [ ]:
col_name_to_sem_type = get_sportsDB_soccer_type_mappings()

results = {}
for table in get_all_sportsDB_soccer_tables(True):
    print(table)
    results[table.split(".csv")[0]] = {}
    col_names = load_sportsDB_soccer_table(table, True)
    for col_num, col_name in enumerate(col_names):
        try:
            results[table.split(".csv")[0]][f"column_{col_num}"] = { "semanticType": col_name_to_sem_type[col_name]}
        except:
            print(col_name)
    
with open(join(os.environ["SPORTS_DB"], "Soccer", "sportsDB_type_sportsDB.json"), "w") as f:
    json.dump(results, f)